In [1]:
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, Trainer
#from peft import LoraConfig, PeftModel
from trl import SFTTrainer


/home/user/conda/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
import torch

a = torch.FloatTensor(120, 768, 3072)

print (torch.numel(a)*a.element_size()/2024./2024.)

276.44112546673125


In [3]:
a

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [4]:
import pickle

In [5]:
with open('llama.pickle', 'rb') as handle:
       UV_dict = pickle.load(handle)

In [6]:
UV_dict.keys()

dict_keys(['up', 'down'])

In [7]:
UV_dict['up'][0].shape

torch.Size([4096, 4096])

In [8]:
!pip install scipy

In [9]:
DEVICE = "cuda:3"

In [10]:
import torch
if torch.cuda.is_available():
  print ("0")
else:
  print ("1")


0


In [11]:
base_model_name = "TheBloke/Llama-2-7B-fp16"
refined_model = "7b_opst"
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True, max_length=512, truncation=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "left",  # Fix for fp16
# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)
# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map=DEVICE,torch_dtype=torch.bfloat16
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
type(base_model.model.layers[3].mlp.down_proj.bias)

NoneType

In [13]:
base_model.model.layers

ModuleList(
  (0-31): 32 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
      (act_fn): SiLUActivation()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)

In [7]:
llama_tokenizer.add_special_tokens({
    "eos_token": llama_tokenizer.convert_ids_to_tokens(base_model.config.eos_token_id),
    "bos_token": llama_tokenizer.convert_ids_to_tokens(base_model.config.bos_token_id),
    "unk_token": llama_tokenizer.convert_ids_to_tokens(
        base_model.config.pad_token_id if base_model.config.pad_token_id != -1 else tokenizer.pad_token_id
    ),
})

3

In [8]:
# Dataset
from custom_dataset import get_custom_dataset
training_data = get_custom_dataset(llama_tokenizer, split="train")
val_data = get_custom_dataset(llama_tokenizer, split="validation")

Found cached dataset parquet (/root/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-2960c57d7e52ab15/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Map:   0%|          | 0/84437 [00:00<?, ? examples/s]

Filter:   0%|          | 0/84437 [00:00<?, ? examples/s]

Parameter 'function'=<function get_custom_dataset.<locals>.<lambda> at 0x7fa1653ff550> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Map:   0%|          | 0/2241 [00:00<?, ? examples/s]

Map:   0%|          | 0/2241 [00:00<?, ? examples/s]

Map:   0%|          | 0/2241 [00:00<?, ? examples/s]

In [ ]:
val_data[5].keys()

## Freeze parts of model

In [9]:
for param in base_model.parameters():
    param.requires_grad = False

In [10]:
for i in range(len(base_model.model.layers), 3):
    for param in base_model.model.layers[i].mlp.parameters():
        param.requires_grad = True


## Add lora

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for pname, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            print(pname, param.numel())
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
from peft import LoraConfig, get_peft_model

rank = 8
target_modules = ['gate_proj', "q_proj", "v_proj", "k_proj", "o_proj"]
ignore_modules = []
# target_modules =  ["q_proj", "v_proj", "k_proj", "o_proj"]
# ignore_modules = ['down_proj', 'up_proj', 'gate_proj']
# target_modules = ['down_proj', 'up_proj', 'gate_proj']
# ignore_modules =  ["q_proj", "v_proj", "k_proj", "o_proj"]
alpha = 8
init = 'lora'
config = LoraConfig(
    r=rank,
    lora_alpha=alpha, 
    # target_modules=['k_proj', 'o_proj', 'down_proj', 'q_proj', 'up_proj', 'gate_proj', 'v_proj'], 
    target_modules=target_modules,
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

In [13]:
# rank 8: trainable params: 8388608 || all params: 3508809728 || trainable%: 0.23907275259355415
# rank 64: trainable params: 67108864 || all params: 3567529984 || trainable%: 1.8811016109458436
# rank 1: trainable params: 1048576 || all params: 3501469696 || trainable%: 0.029946739256314844
# rank 16: trainable params: 16777216 || all params: 3517198336 || trainable%: 0.47700511592645084
model = get_peft_model(base_model, config)
print_trainable_parameters(base_model)

model.layers.0.self_attn.q_proj.lora_A.default.weight 32768
model.layers.0.self_attn.q_proj.lora_B.default.weight 32768
model.layers.0.self_attn.k_proj.lora_A.default.weight 32768
model.layers.0.self_attn.k_proj.lora_B.default.weight 32768
model.layers.0.self_attn.v_proj.lora_A.default.weight 32768
model.layers.0.self_attn.v_proj.lora_B.default.weight 32768
model.layers.0.self_attn.o_proj.lora_A.default.weight 32768
model.layers.0.self_attn.o_proj.lora_B.default.weight 32768
model.layers.0.mlp.gate_proj.lora_A.default.weight 32768
model.layers.0.mlp.gate_proj.lora_B.default.weight 88064
model.layers.1.self_attn.q_proj.lora_A.default.weight 32768
model.layers.1.self_attn.q_proj.lora_B.default.weight 32768
model.layers.1.self_attn.k_proj.lora_A.default.weight 32768
model.layers.1.self_attn.k_proj.lora_B.default.weight 32768
model.layers.1.self_attn.v_proj.lora_A.default.weight 32768
model.layers.1.self_attn.v_proj.lora_B.default.weight 32768
model.layers.1.self_attn.o_proj.lora_A.default

In [14]:
base_model.device

device(type='cuda', index=0)

In [15]:
len(val_data[6]['input_ids'])

512

In [16]:
trainer_config =  {
    "evaluation_strategy": "steps",
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 64,
    "eval_steps": 75,
    # "eval_steps": 3,
    "save_steps": 75,
    # "save_steps": 3,
    "logging_steps": 5,
    "learning_rate": 0.0003,
    "lr_scheduler_type": "cosine",
    "warmup_steps": 50,
    # "warmup_steps": 5,
    "num_train_epochs": 3,
    # "max_steps": 15,
    "fp16": False,
    "bf16": True,
    "torch_compile": False,
    "optim": "adamw_torch"
}

training_args = TrainingArguments(
        output_dir="/home/jovyan/team_code/big_models",
        save_total_limit=5,
        logging_first_step=True,
        eval_delay=0,
        seed=42,
        data_seed=42,
        load_best_model_at_end=True,
        report_to=None,
        ddp_find_unused_parameters=None,
        # deepspeed=deepspeed_config,
        **trainer_config
    )
training_args.local_rank = -1

In [17]:
training_args.device

device(type='cuda', index=0)

In [18]:
base_model.device

device(type='cuda', index=0)

In [20]:
from transformers import TrainerCallback, TrainerCallback, TrainerState, TrainerControl

class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        
        peft_model_path = "/home/jovyan/team_code/big_models2/"
        
        kwargs["model"].save_pretrained(peft_model_path)
        return control

callbacks = [SavePeftModelCallback]

In [22]:

training_args = TrainingArguments(
        output_dir="/home/jovyan/team_code/big_models",
        save_total_limit=5,
        local_rank = 1,
        logging_first_step=True,
        eval_delay=0,
        seed=42,
        data_seed=42,
        load_best_model_at_end=True,
        report_to=None,
        ddp_find_unused_parameters=None,
        **trainer_config
    )
training_args.local_rank = -1


trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=training_data,
        callbacks = callbacks,
        eval_dataset=val_data,
        tokenizer=llama_tokenizer)

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/user/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


In [ ]:
fine_tuning.model.save_pretrained("/home/jovyan/team_code/big_models/llama-2-7b_8bit.pth")

In [ ]:
len(training_data)

In [ ]:
11110MiB used